# SETUP

## imports

In [3]:
import string
import numpy as np
from itertools import cycle
import requests


## constants

In [14]:
lowercase=string.ascii_lowercase

## helpers

In [30]:
def get_level_input(lvl_num):
    with open('advent_inputs/%d.txt' %lvl_num) as f:
        level_input=f.read()
        return level_input[:-1]

# LEVEL 1

## setup

In [38]:
polymers=get_level_input(1)

FileNotFoundError: [Errno 2] No such file or directory: 'advent_inputs/1.txt'

## part one

In [36]:
polymer_temp=polymers
i=0
while i!=len(polymer_temp)-1:
    if(abs(ord(polymer_temp[i])-ord(polymer_temp[i+1]))==32):
        polymer_temp=polymer_temp[0:i]+polymer_temp[i+2:]
        i=max(0,i-1)
    else:
        i+=1
print(len(polymer_temp))

9202


## part two

In [37]:
scores=[]
for letter in lowercase:
    polymer_temp=polymers
    i=0
    polymer_temp=polymer_temp.translate(str.maketrans('', '', letter+letter.upper()))
    while i!=len(polymer_temp)-1:
        if(abs(ord(polymer_temp[i])-ord(polymer_temp[i+1]))==32):
            polymer_temp=polymer_temp[0:i]+polymer_temp[i+2:]
            i=max(0,i-1)
        else:
            i+=1
    scores.append(len(polymer_temp))
print(min(scores))

6394


# LEVEL 2

## setup

In [31]:
polymers=get_level_input(2)

## part one

In [36]:
polymer_temp=polymers
i=0
while i!=len(polymer_temp)-1:
    if(abs(ord(polymer_temp[i])-ord(polymer_temp[i+1]))==32):
        polymer_temp=polymer_temp[0:i]+polymer_temp[i+2:]
        i=max(0,i-1)
    else:
        i+=1
print(len(polymer_temp))

9202


## part two

In [37]:
scores=[]
for letter in lowercase:
    polymer_temp=polymers
    i=0
    polymer_temp=polymer_temp.translate(str.maketrans('', '', letter+letter.upper()))
    while i!=len(polymer_temp)-1:
        if(abs(ord(polymer_temp[i])-ord(polymer_temp[i+1]))==32):
            polymer_temp=polymer_temp[0:i]+polymer_temp[i+2:]
            i=max(0,i-1)
        else:
            i+=1
    scores.append(len(polymer_temp))
print(min(scores))

6394


# LEVEL 3

## setup

In [75]:
fabric_list=get_level_input(3).splitlines()

In [76]:
fabric_list=[x.split(" ") for x in fabric_list]

In [77]:
fabric_list[0]

['#1', '@', '871,327:', '16x20']

## part one

In [78]:
fabric=np.zeros((1000,1000))
for i in fabric_list:
    dim=tuple([int(z) for z in i[2][:-1].split(",")])
    size=tuple([int(z) for z in i[3].split("x")])
    fabric[dim[0]:dim[0]+size[0],dim[1]:dim[1]+size[1]]+=1
    NUM_SINGLE_USED=(fabric > 1).sum()
print(NUM_SINGLE_USED)

101565


## part two

In [80]:
for i in fabric_list:
    dim=tuple([int(z) for z in i[2][:-1].split(",")])
    size=tuple([int(z) for z in i[3].split("x")])
    if((fabric[dim[0]:dim[0]+size[0],dim[1]:dim[1]+size[1]]==1).all()):
        print(i[0])

#656


# LEVEL 4

## setup

In [31]:
polymers=get_level_input(4)

## part one

In [36]:
polymer_temp=polymers
i=0
while i!=len(polymer_temp)-1:
    if(abs(ord(polymer_temp[i])-ord(polymer_temp[i+1]))==32):
        polymer_temp=polymer_temp[0:i]+polymer_temp[i+2:]
        i=max(0,i-1)
    else:
        i+=1
print(len(polymer_temp))

9202


## part two

In [37]:
scores=[]
for letter in lowercase:
    polymer_temp=polymers
    i=0
    polymer_temp=polymer_temp.translate(str.maketrans('', '', letter+letter.upper()))
    while i!=len(polymer_temp)-1:
        if(abs(ord(polymer_temp[i])-ord(polymer_temp[i+1]))==32):
            polymer_temp=polymer_temp[0:i]+polymer_temp[i+2:]
            i=max(0,i-1)
        else:
            i+=1
    scores.append(len(polymer_temp))
print(min(scores))

6394


# LEVEL 5

## setup

In [31]:
polymers=get_level_input(5)

## part one

In [36]:
polymer_temp=polymers
i=0
while i!=len(polymer_temp)-1:
    if(abs(ord(polymer_temp[i])-ord(polymer_temp[i+1]))==32):
        polymer_temp=polymer_temp[0:i]+polymer_temp[i+2:]
        i=max(0,i-1)
    else:
        i+=1
print(len(polymer_temp))

9202


## part two

In [37]:
scores=[]
for letter in lowercase:
    polymer_temp=polymers
    i=0
    polymer_temp=polymer_temp.translate(str.maketrans('', '', letter+letter.upper()))
    while i!=len(polymer_temp)-1:
        if(abs(ord(polymer_temp[i])-ord(polymer_temp[i+1]))==32):
            polymer_temp=polymer_temp[0:i]+polymer_temp[i+2:]
            i=max(0,i-1)
        else:
            i+=1
    scores.append(len(polymer_temp))
print(min(scores))

6394
